# GAPy for model parameters discovery

The GAPy framework is used fo optimize the parameters of a model by using a dataet containing
- model inputs
- model (noisy) outputs
- the model generic form


In [10]:
import numpy as np
from gapy import GAPy,selection_rank

from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

Loading BokehJS ...

## Synthetic data creation

The model is implemented in the `mysterious_function` function.

$A \cdot sin(B \cdot (x-C)) + D$

where A, B, C and D are the model parameters to *discover*. Let's suppose that these parameters vary in the range [0,5].

`x_values` is the model input variable. `NUM_POINTS` samples are created and the model is used to generate the corrsponding outputs `y_pure`.
To make the thing more interesting some noise is added to `y_pure`: so `y_values` is finally created by adding a random noise whose percentual maximum value is `NOISE_PCT`.

Model original parameters are in the `actual_pars` dictionary.

In [4]:
NOISE_PCT=10
NUM_POINTS=50

def mysterious_function(x,pars):
    # A sin (B(x-C)) + D
    collector=[]
    for xi in x:
        collector.append(pars['A']*np.sin(pars['B']*(xi+pars['C']))+pars['D'])
    return np.array(collector)
        
        
x_values=np.linspace(-10,10,num=NUM_POINTS)

actual_pars={'A':1,
             'B':.75,
             'C':2,
             'D':1}

y_pure=mysterious_function(x_values, actual_pars)
y_values=np.array([yi+np.random.randn()*NOISE_PCT/100*yi for yi in y_pure])

## Setting up the GAPy

### Initial population

First step is to decide how to code the choromosomes. GAPy is very flexible on it. This is basically a *real-coded* GA problem but here we use a dictionary (similar to `actual_pars`) so we can directly reuse the `misteryous_function` that takes a dictionary as parameters of our function.

The `initial_population` is thus created as a list of dictionaries. Each element of the dictionary is set at random in the parameters range (if we know it...)

In [5]:
NUM_CROMOSOMES=100

# initial population
initial_population=[]
for i in range(NUM_CROMOSOMES):
    initial_population.append({'A':np.random.randint(0,5),
                               'B':np.random.randint(0,5),
                               'C':np.random.randint(0,5),
                               'D':np.random.randint(0,5)})

### Fitness function

GAPy supports fitness functios with one only input parameter: the solution to evaluate.

In this case `fit_fun_params_tuning` takes more parameters to consider the *training dataset*: when it will be passed to the optimizer we will handle this issue...

In [6]:
# FITNESS FUNCTION
def fit_fun_params_tuning(solution,xvalues,y_actual,base_function):
    y_pred=base_function(xvalues,solution)
    return np.mean(np.abs(y_pred-y_actual))

### Genetic operators

- crossover function
- mutation function

> `crossover_weighted_params_tuning` uses 4 input arguments instead of the *standard* two parents. GAPy supports this approach **provided** the two additional arguments are the fitness values of the two parents.

In [7]:
# MUTATION
def mutation_params_tuning(solution):
    # one param is modified at random in the range +/- 0.25
    # choose which param to modify
    muted_param=np.random.choice(['A','B','C','D'])
    # modify it
    solution[muted_param]=solution[muted_param]+np.random.randn()*0.5
    return solution


# CROSSOVER
def crossover_basic_params_tuning(s1,s2):
    # gene-wise average of the two parents
    res={}
    for param in ['A','B','C','D']:
        res[param]=(s1[param]+s2[param])/2
    return res

def crossover_weighted_params_tuning(s1,s2,fit1,fit2):
    # weighted average: the better the solution the higher the weight
    w1=1-fit1/(fit1+fit2)
    w2=1-w1
    res={}
    for param in ['A','B','C','D']:
        res[param]=w1*s1[param]+w2*s2[param]
    return res

## Initializing the GAPy optimizer

> Note that the fitness function is adapted through the `lambda` operator to a 1-argument function by passing the data and the generic model.

In [8]:
params_optimizer=GAPy(initial_population,
          selection_function=selection_rank,
          fitness_function=lambda x:fit_fun_params_tuning(x,x_values,y_values,mysterious_function),
          crossover_function=crossover_weighted_params_tuning,
          mutation_function=mutation_params_tuning)


## Optimization

In [9]:
soluzione=params_optimizer.optimize(max_generations=600,
                                    elite=3,
                                    max_time=10,
                                    patience=100,
                                    print_frequency=10)


[1/600] Ave:   1.91  Curr.Best:  0.612  Best:  0.612 [Age: 0]
[    10/600   ] Ave.:   0.6 Curr.Best: 0.484 Best: 0.484 [AGE 2]
[    20/600   ] Ave.:  0.59 Curr.Best: 0.484 Best: 0.484 [AGE 12]
[    30/600   ] Ave.:  0.59 Curr.Best: 0.265 Best: 0.265 [AGE 0]
[    40/600   ] Ave.: 0.594 Curr.Best: 0.237 Best: 0.237 [AGE 0]
[    50/600   ] Ave.: 0.223 Curr.Best: 0.147 Best: 0.147 [AGE 1]
[    60/600   ] Ave.:  0.14 Curr.Best: 0.103 Best: 0.103 [AGE 0]
[    70/600   ] Ave.: 0.122 Curr.Best:0.0963 Best:0.0963 [AGE 0]
[    80/600   ] Ave.: 0.115 Curr.Best:0.0905 Best:0.0905 [AGE 0]
[    90/600   ] Ave.: 0.121 Curr.Best:0.0894 Best:0.0894 [AGE 0]
[   100/600   ] Ave.: 0.108 Curr.Best:0.0894 Best:0.0894 [AGE 0]
[   110/600   ] Ave.: 0.117 Curr.Best:0.0894 Best:0.0894 [AGE 9]
[   120/600   ] Ave.: 0.108 Curr.Best:0.0894 Best:0.0894 [AGE 19]
[   130/600   ] Ave.: 0.114 Curr.Best:0.0893 Best:0.0893 [AGE 1]
[   140/600   ] Ave.: 0.125 Curr.Best:0.0893 Best:0.0893 [AGE 11]
[   150/600   ] Ave.: 0.1

## Evaluation

The solution is tested. A GAPy solution has two components: `.coded` contains the solution according to the initial coding, decided by the user (inthis case the dictionary).

- the model output values corresponding to the input variables in the dataset are calculated
- then they are compared to the **noisy** and **non noisy** data


In [12]:
# the model output values corresponding to the input variables in the dataset are calculated
y_pred=mysterious_function(x_values,soluzione.coded)

# then they are compared to the noisy and non noisy data
p = figure(title='Function fitting results',
           background_fill_color="#fafafa",           
           width=1000,height=600)
p.line(x=x_values,
       y=y_pure,
       line_width=4,
       legend_label='Theoretical')
p.scatter(x=x_values,
         y=y_values,
       legend_label='Actual (noisy)',      
       color='red',alpha=0.6,size=10,marker='triangle')
p.scatter(x=x_values,
         y=y_pred,
       legend_label='Estimated points',      
       color='green',alpha=0.6,size=10)
p.legend.click_policy="hide"
show(p)

